In [2]:
import requests
import json
from pprint import pprint
import pandas as pd
import time
from pymongo import MongoClient

In [3]:
api_key = "aeRc3h5tR5wweegUs1yyQ98QGs1H4ti2jpGW5hGn"
base_url = "https://api.nasa.gov/neo/rest/v1/feed"
start_date = "2020-01-01"
end_date = "2029-12-31"
iterations = 7

all_neo_data = []

url = f"{base_url}?start_date={start_date}&end_date={end_date}&api_key={api_key}"

for _ in range(iterations):
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        for date, neos in data["near_earth_objects"].items():
            for neo in neos:
                all_neo_data.append({
                    "id": neo["id"],
                    "name": neo["name"],
                    "absolute_magnitude_h": neo["absolute_magnitude_h"],
                    "estimated_diameter_km_min": neo["estimated_diameter"]["kilometers"]["estimated_diameter_min"],
                    "estimated_diameter_km_max": neo["estimated_diameter"]["kilometers"]["estimated_diameter_max"],
                    "is_potentially_hazardous": neo["is_potentially_hazardous_asteroid"],
                    "close_approach_date": neo["close_approach_data"][0]["close_approach_date"] if neo["close_approach_data"] else None,
                    "relative_velocity_kph": neo["close_approach_data"][0]["relative_velocity"]["kilometers_per_hour"] if neo["close_approach_data"] else None,
                    "miss_distance_km": neo["close_approach_data"][0]["miss_distance"]["kilometers"] if neo["close_approach_data"] else None,
                    "orbiting_body": neo["close_approach_data"][0]["orbiting_body"] if neo["close_approach_data"] else None
                })

        url = data["links"].get("next")
        time.sleep(0.5)
        if not url:
            break
    else:
        print("Error fetching data:", response.status_code, response.text)
        break

df = pd.DataFrame(all_neo_data)
df.head()
df.to_csv("../Resources/neo_data.csv", index=False)

Error fetching data: 400 {"code":400,"http_error":"BAD_REQUEST","error_message":"Date Format Exception - Expected format (yyyy-mm-dd) - The Feed date limit is only 7 Days","request":"http://api.nasa.gov/rest/v1/feed?start_date=2020-01-01&end_date=2029-12-31"}


OSError: Cannot save file into a non-existent directory: '..\Resources'

In [ ]:
# Create .js Data File

df.shape[0]
with open("neo_data.js", "w") as js_file:
    js_file.write("const neoData = ")
    json.dump(all_neo_data, js_file, indent=2)
    js_file.write(";\nexport default neoData;")

In [ ]:
# Write data to MongoDB

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")  # Change if using a remote server
db = client["nasa_neo"]  # Database name
collection = db["neo_data"]  # Collection name

# Insert data into MongoDB
if all_neo_data:
    collection.insert_many(all_neo_data)
    print("Data uploaded successfully to MongoDB")
else:
    print("No data to upload")

# Verify upload
print(f"Total documents in collection: {collection.count_documents({})}")

Data uploaded successfully to MongoDB
Total documents in collection: 1338
